In [11]:
import pandas as pd
import numpy as np
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import math
import time
import os
import ast
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
from plotly.subplots import make_subplots
import sys
sys.path.append('../')
from defillama_utils import *
sys.path.pop()
# import defillama_utils

nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

chains_df = pd.DataFrame(np.array([
        # ['Optimism','2021-06-21'],
        # ['Arbitrum','2021-08-26'],
        # ['Polygon','2019-01-01'],
        # ['Solana','2019-01-01'],
        # ['Ethereum','2019-01-01'],
        # ['Binance','2019-01-01'],
        # ['Avalanche','2019-01-01'],
        ['Fantom','2019-01-01'],
        ['Harmony','2019-01-01'], #just weird
        # ['xDai','2019-01-01'],
        # ['Terra','2019-01-01']
        ]),columns=['chain','launch_date']
)       

tvl_protocol_threshold = 0 #1_000_000
# tvl_protocol_threshold_max = 10_000_000

trailing_num_days = 10000

floor_round = 5
floor_round_tvl = 1_000_000_000 #1_000_000_000 = 1b

filter_to_chains = 1 # if 1, filter to this chain list, if 0 then don't
exclude_ethereum_l1 = 0 #If 1, filter out Eth L1
chain_lbl = 'all'

start_date = date.today()-timedelta(days=trailing_num_days +1)
manual_chains_list = chains_df['chain'].tolist()
print(start_date)


1995-06-29


In [12]:
excl_str_list = ['-borrowed','-staking','-pool2']
dne_str_list = ['borrowed','staking','pool2','treasury']
###get all apps###

all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
#filter to ones that are on Optimism
protocols = res[['slug','chainTvls']]
# filter to min tvl
res = res[res['tvl'] > 0] ##greater than X


In [13]:

# re = res['chainTvls']
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
display(protocols)
#If one chain # protocols = protocols[protocols['chainTvls'].map(set(chains_list).issubset)]

# Limit to our subset of chains

for str_e in excl_str_list:
        protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: [cn for cn in x if cn.find(str_e) == -1] )
for str_dne in dne_str_list:
        protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: [cn for cn in x if cn != str_dne] )

op_protocols = protocols[protocols['chainTvls'].apply(lambda x: any(y in x for y in ['Optimism']) )]

if filter_to_chains == 1:
        protocols = protocols[protocols['chainTvls'].apply(lambda x: any(y in x for y in manual_chains_list) )]
        solp = protocols[protocols['chainTvls'].apply(lambda x: any(y in x for y in 'Solana') )]
        #generate unfiltered list
        protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: [cn for cn in x if cn in manual_chains_list] )
        chain_lbl = 'filtered'


protocols = protocols[protocols['slug'] != 'polygon-bridge-&-staking']
# display(solp)

,slug,chainTvls
0,binance-cex,"[Binance, Ethereum, Tron, Bitcoin]"
1,polygon-bridge-&-staking,[Polygon]
2,makerdao,[Ethereum]
3,lido,"[Ethereum, Moonriver, Solana, Terra, Moonbeam]"
4,okx,"[Ethereum, Avalanche, Polygon, Tron, Arbitrum,..."
...,...,...
2099,ora-finance,"[Aurora-staking, Aurora, pool2, Aurora-pool2, ..."
2100,starfish-finance,"[Astar, Astar-staking, staking]"
2101,hera-finance,"[Metis, Metis-staking, staking]"
2102,dexible,[Ethereum]


In [14]:
chains_list = protocols['chainTvls'].explode('chainTvls')
# chains_list = chains_list = chains_df['chain'].tolist()
chains_list = chains_list.drop_duplicates()
#get unique chains
chains_list = chains_list.to_list()
print(str(len(chains_list)))
chains_list

5


['Avalanche', 'Polygon', 'Fantom', 'Optimism', 'Harmony']

In [15]:
# get chain tvls

chains_resp = get_chain_tvls(chains_list)
if filter_to_chains ==1 :
        chains_resp = chains_resp.merge(chains_df, on ='chain')
        chains_resp = chains_resp[chains_resp['date'] >= chains_resp['launch_date']]
        chains_resp = chains_resp.drop(columns=['launch_date'])
if exclude_ethereum_l1 ==1:
        chains_resp = chains_resp[chains_resp['chain']!='Ethereum']

In [16]:
display(chains_resp)

,date,totalLiquidityUSD,chain
0,2020-12-20,0.000000e+00,Avalanche
1,2021-02-03,1.244221e+02,Avalanche
2,2021-02-04,3.612846e+06,Avalanche
3,2021-02-05,4.860423e+06,Avalanche
4,2021-02-06,6.220490e+06,Avalanche
...,...,...,...
3149,2022-11-11,2.099861e+07,Harmony
3150,2022-11-12,2.069702e+07,Harmony
3151,2022-11-13,2.081718e+07,Harmony
3152,2022-11-14,2.076766e+07,Harmony


In [17]:
chains = chains_resp.copy()
all_chains = chains.groupby('date').sum('totalLiquidityUSD')
all_chains = all_chains.reset_index()
# display(all_chains)
chains['last_day_tvl'] = chains.groupby('chain')['totalLiquidityUSD'].shift(1)
chains['last_day_tvl'].fillna(0,inplace=True)

chains['month'] = chains['date'] + pd.offsets.MonthBegin(-1)
chains['quarter'] = pd.PeriodIndex(chains['date'], freq='Q')

tp = ['month','quarter']

dfl = []
for t in tp:

        month_chains = chains.groupby(['chain',t]).agg({'totalLiquidityUSD':'first'})
        month_chains = month_chains.rename(columns={'totalLiquidityUSD':'start_of_'+t+'_tvl'})
        month_chains[t + 'ly_tvl_share'] = (month_chains['start_of_'+t+'_tvl'] 
                                / month_chains.groupby(t)['start_of_'+t+'_tvl'].transform('sum'))*100
        month_chains.reset_index(inplace=True)
        # month_chains[t] = t
        dfl.append(month_chains)

for df_t in dfl:
        for t in tp:
                if t in df_t.columns:
                        chains = chains.merge(df_t,on=['chain',t],how = 'left')
# display(chains)

In [18]:
display(all_chains)

,date,totalLiquidityUSD
0,2020-10-09,2.495342e+03
1,2020-10-10,4.867179e+04
2,2020-10-11,1.428954e+05
3,2020-10-12,6.532866e+05
4,2020-10-13,6.659858e+05
...,...,...
763,2022-11-11,3.948738e+09
764,2022-11-12,3.842578e+09
765,2022-11-13,3.745208e+09
766,2022-11-14,3.636174e+09


In [19]:
# display(month_chains)
# chains = chains.merge(month_chains,on=['chain','month'],how = 'left')
# chains['start_month_tvl'] = chains.groupby(['chain','month']).agg('totalLiquidityUSD':'first')
# chains['start_month_tvl'].fillna(0,inplace=True)

# chains['all_chains_last_day_tvl'] = chains.groupby('date')['last_day_tvl'].transform('sum')
chains['chains_tvl_share'] = (chains['last_day_tvl'] / chains.groupby('date')['last_day_tvl'].transform('sum'))*100
chains['chains_tvl_share'].fillna(100,inplace=True)
display(chains)
# display(chains.sort_values(by=['date'], ascending = True))

current_tvl = chains[chains['date'] == pd.to_datetime("today").strftime("%Y-%m-%d") ]
display(current_tvl)

,date,totalLiquidityUSD,chain,last_day_tvl,month,quarter,start_of_month_tvl,monthly_tvl_share,start_of_quarter_tvl,quarterly_tvl_share,chains_tvl_share
0,2020-12-20,0.000000e+00,Avalanche,0.000000e+00,2020-12-01,2020Q4,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
1,2021-02-03,1.244221e+02,Avalanche,0.000000e+00,2021-02-01,2021Q1,1.244221e+02,0.002427,1.244221e+02,0.011212,0.000000
2,2021-02-04,3.612846e+06,Avalanche,1.244221e+02,2021-02-01,2021Q1,1.244221e+02,0.002427,1.244221e+02,0.011212,0.002724
3,2021-02-05,4.860423e+06,Avalanche,3.612846e+06,2021-02-01,2021Q1,1.244221e+02,0.002427,1.244221e+02,0.011212,44.436318
4,2021-02-06,6.220490e+06,Avalanche,4.860423e+06,2021-02-01,2021Q1,1.244221e+02,0.002427,1.244221e+02,0.011212,51.801904
...,...,...,...,...,...,...,...,...,...,...,...
3149,2022-11-11,2.099861e+07,Harmony,2.029018e+07,2022-11-01,2022Q4,2.340618e+07,0.489745,2.900055e+07,0.576933,0.529527
3150,2022-11-12,2.069702e+07,Harmony,2.099861e+07,2022-11-01,2022Q4,2.340618e+07,0.489745,2.900055e+07,0.576933,0.531780
3151,2022-11-13,2.081718e+07,Harmony,2.069702e+07,2022-11-01,2022Q4,2.340618e+07,0.489745,2.900055e+07,0.576933,0.538623
3152,2022-11-14,2.076766e+07,Harmony,2.081718e+07,2022-11-01,2022Q4,2.340618e+07,0.489745,2.900055e+07,0.576933,0.555835


,date,totalLiquidityUSD,chain,last_day_tvl,month,quarter,start_of_month_tvl,monthly_tvl_share,start_of_quarter_tvl,quarterly_tvl_share,chains_tvl_share
651,2022-11-15,1.136900e+09,Avalanche,1.119127e+09,2022-11-01,2022Q4,1.625630e+09,34.014290,1.938558e+09,38.565438,30.777609
1419,2022-11-15,1.283743e+09,Polygon,1.268953e+09,2022-11-01,2022Q4,1.419182e+09,29.694619,1.441189e+09,28.670838,34.898024
2046,2022-11-15,5.166942e+08,Fantom,5.058137e+08,2022-11-01,2022Q4,6.235641e+08,13.047302,6.046569e+08,12.028968,13.910602
2536,2022-11-15,6.803669e+08,Optimism,7.215123e+08,2022-11-01,2022Q4,1.087474e+09,22.754045,1.013268e+09,20.157823,19.842624
3153,2022-11-15,1.954044e+07,Harmony,2.076766e+07,2022-11-01,2022Q4,2.340618e+07,0.489745,2.900055e+07,0.576933,0.571141


In [20]:
display(protocols)

,slug,chainTvls
4,okx,"[Avalanche, Polygon]"
5,bitfinex,"[Avalanche, Fantom, Polygon]"
7,curve,"[Optimism, Harmony, Avalanche, Polygon, Fantom]"
8,aave-v2,"[Avalanche, Polygon]"
9,uniswap-v3,"[Optimism, Polygon]"
...,...,...
2074,wolf-safe-poor-people,[Polygon]
2079,champion-finance,[Avalanche]
2080,dawn-star-finance,[Polygon]
2094,based-next-generation,[Fantom]


In [21]:
#get detail by app
# from defillama_utls
# df_df = defillama_utils.get_range(protocols)
df_df_raw = get_range(protocols)

# op_df_df_raw = get_range(op_protocols) #get data for protocols on Optimism



In [ ]:
df_df = df_df_raw.copy()
# op_df_df = op_df_df_raw.copy()
# display(df_df[(df_df['protocol'] != 'renvm') & (df_df['chain'] == 'Solana')].sort_values(by='date'))

In [ ]:
# Keep last whole day only

df_df = df_df[df_df['total_app_tvl'] >0] #make sure we're getting the first day with TVL

df_df = df_df[df_df['date'] == df_df['date'].dt.floor('d')]

df_df.drop_duplicates(inplace=True)



In [ ]:

# op_df_sum = op_df_sum[\
#                         (~op_df_sum['chain'].str.contains('-borrowed')) &\
#                         (~op_df_sum['chain'].str.contains('-staking')) &\
#                         (~op_df_sum['chain'].str.contains('-pool2')) &\
#                         (~op_df_sum['chain'].str.contains('-treasury')) &\
#                         (~( op_df_sum['chain'] == 'treasury') ) &\
#                         (~( op_df_sum['chain'] == 'borrowed') ) &\
#                         (~( op_df_sum['chain'] == 'staking') ) &\
#                         (~( op_df_sum['chain'] == 'pool2') )       
# #                         & (~( netdf_df['chain'] == 'Ethereum') )
# ]
# display(op_df_sum[op_df_sum['chain'] == 'Ethereum-treasury'])

In [ ]:
# OP get start date across chains
# Filter out L1
op_df_sum = op_df_df[op_df_df['chain'] != 'Ethereum']
op_df_sum = remove_bad_cats(op_df_sum)
# op_df_df.head()
op_df_sum = op_df_sum.groupby(['protocol','chain','date','category']).agg({'usd_value':'sum','total_app_tvl':'mean'})
op_df_sum = op_df_sum.reset_index()
op_df_sum = op_df_sum.sort_values(by='date',ascending=True)
op_df_sum = op_df_sum.groupby(['protocol','chain','category']).agg({'usd_value':'last','date':'min','total_app_tvl':'last'})
#get map across deployments
op_df_sum_max = op_df_sum.groupby(['protocol','category']).agg({'total_app_tvl':'max'})
op_df_sum_max = op_df_sum_max.reset_index()
op_df_sum_max = op_df_sum_max.rename(columns={'total_app_tvl':'max_app_tvl'})
# display(op_df_sum_max)

# op_df_sum['max_app_tvl'] = 
op_df_sum = op_df_sum.reset_index()

op_df_sum = op_df_sum.merge(op_df_sum_max, on=['protocol'],how='left')
# display(op_df_sum)
op_df_sum = op_df_sum.rename(columns={'usd_value':'current_tvl_usd','date':'launch_date'})
#generate OP launch dates
op_launch = op_df_sum[op_df_sum['chain'] == 'Optimism']
op_launch = op_launch.rename(columns={'launch_date':'op_launch','total_app_tvl':'op_tvl_usd'})
op_launch = op_launch[['protocol','op_launch','op_tvl_usd']]

op_df_sum = op_df_sum.merge(op_launch,on='protocol')
op_df_sum['chain_id'] = op_df_sum['protocol'] + ' - ' + op_df_sum['chain']

op_df_sum.loc[op_df_sum.chain!='Optimism', 'days_after_op'] = (op_df_sum['launch_date'] - op_df_sum['op_launch']).dt.days

#Filter TVL > threshold
op_df_sum = op_df_sum[(op_df_sum['max_app_tvl'] > tvl_protocol_threshold) | (op_df_sum['chain'] == 'Optimism')]
#Filter > OP Mainnet Launch
# op_df_sum = op_df_sum[op_df_sum['launch_date'] > '2021-06-26']

display(op_df_sum[op_df_sum['protocol'] == 'ooki'])


In [ ]:
# OP get start date across chains
# Filter out L1
op_df_sum = op_df_df[op_df_df['chain'] != 'Ethereum']
op_df_sum = remove_bad_cats(op_df_sum)
# op_df_df.head()
op_df_sum = op_df_sum.groupby(['protocol','chain','date','category']).agg({'usd_value':'sum','total_app_tvl':'mean'})
op_df_sum = op_df_sum.reset_index()
op_df_sum = op_df_sum.sort_values(by='date',ascending=True)
op_df_sum = op_df_sum.groupby(['protocol','chain','category']).agg({'usd_value':'last','date':'min','total_app_tvl':'mean'})
op_df_sum = op_df_sum.reset_index()
op_df_sum = op_df_sum.rename(columns={'usd_value':'current_tvl_usd','date':'launch_date'})
#generate OP launch dates
op_launch = op_df_sum[op_df_sum['chain'] == 'Optimism']
op_launch = op_launch.rename(columns={'launch_date':'op_launch','total_app_tvl':'op_tvl_usd'})
op_launch = op_launch[['protocol','op_launch','op_tvl_usd']]

op_df_sum = op_df_sum.merge(op_launch,on='protocol')
op_df_sum['chain_id'] = op_df_sum['protocol'] + ' - ' + op_df_sum['chain']

op_df_sum.loc[op_df_sum.chain!='Optimism', 'days_after_op'] = (op_df_sum['launch_date'] - op_df_sum['op_launch']).dt.days

# op_df_sum['op_status'] = np.where(op_df_sum['days_after_op'].isnull(), 'Optimism Only',
#                                         np.where(op_df_sum['days_after_op'] < -1, 'Other Chain First'
#                                         ,np.where(op_df_sum['days_after_op'] > 1, 'Optimism First'
#                                         ,'Same-Time Deployment'))
#                                         )
# display(op_df_sum[op_df_sum['protocol'] == 'stargate'])


In [ ]:
op_df_sum_agg = op_df_sum.copy()
op_df_sum_agg = op_df_sum_agg[op_df_sum_agg['current_tvl_usd'] > 100_000] #TVL Min
op_df_sum_agg = op_df_sum_agg[~op_df_sum_agg['category'].isin(['Bridge','Cross Chain'])]
# display(op_df_sum_agg['protocol'].drop_duplicates())
op_df_sum_agg['is_optimism'] = 1#np.where(op_df_sum_agg['chain'] == 'Optimism',1,0)
op_df_sum_agg = op_df_sum_agg[['protocol','op_launch','days_after_op','chain','is_optimism']].groupby(['protocol','op_launch']).agg({'days_after_op':'min','is_optimism':'mean'})
op_df_sum_agg = op_df_sum_agg.reset_index()

#re-do for min
op_df_sum_agg['op_status'] = np.where(op_df_sum_agg['days_after_op'].isnull(), 'Optimism Only',
                                        np.where(op_df_sum_agg['days_after_op'] < -2, 'Other Chain First'
                                        ,np.where(op_df_sum_agg['days_after_op'] > 2, 'Optimism First'
                                        ,'Same-Time Deployment'))
                                        )
print(len(op_df_sum_agg))
# display(op_df_sum_agg.sort_values(by='op_status',ascending=True))

op_df_sum_agg_op_first = op_df_sum_agg[op_df_sum_agg['days_after_op'] > -1].sort_values(by=['op_launch','days_after_op'],ascending=[True,False])
op_df_sum_agg_op_first['rolling_days'] = op_df_sum_agg_op_first['days_after_op'].rolling(5, min_periods=1).mean()

# display(op_df_sum_agg_op_first)
opf = px.scatter(op_df_sum_agg_op_first,x='op_launch',y='days_after_op'
                ,hover_data=["protocol"]
                ,labels={
                        "op_launch": "Date Launched on Optimism",
                        "days_after_op": "# Days Until Launched on Other Chain",
                        # "chain": "Chain"
                        }
                ,title="Protocols Launched on Optimism - Time to Deploy on Other Chains (> $100k TVL)\
                                <br><sup><i>Excluding Bridges</i></sup></br>\
                                ")
opf.add_trace(go.Line(x=op_df_sum_agg_op_first['op_launch'],y=op_df_sum_agg_op_first['rolling_days'],name='Trailing 5 Launch Avg',mode='lines'))
opf.show()

op_df_sum_agg['time_period'] = np.where(op_df_sum_agg['op_launch'] < '2022-02-01','Pre-Feb 2022','>= Feb 2022')
display( op_df_sum_agg.groupby('op_status').sum('is_optimism') )
display( op_df_sum_agg.groupby(['time_period','op_status']).sum('is_optimism') )

In [ ]:
#Filter TVL > 1M
op_df_sum = op_df_sum[(op_df_sum['current_tvl_usd'] > 1_000_000) | (op_df_sum['chain'] == 'Optimism')]
#Filter > OP Mainnet Launch


# display(op_df_sum[op_df_sum['protocol'] == 'ooki'])

In [ ]:


op_df_sum_chart = op_df_sum[(op_df_sum['chain'] != 'Optimism') & (~op_df_sum['category'].isin(['Bridge','Cross Chain']))]

op_df_sum_chart = op_df_sum_chart[op_df_sum_chart['launch_date'] > '2021-06-26']
# op_df_sum_chart = op_df_sum_chart[(op_df_sum_chart['launch_date'] > '2021-11-11') & (op_df_sum_chart['op_launch'] > '2021-11-11')] #regenesis
# op_df_sum_chart = op_df_sum_chart[op_df_sum_chart['days_after_op'] > -90]

# op_df_sum_chart['days_after_op'] = np.where(op_df_sum_chart['days_after_op'] < 0 , -1, op_df_sum_chart['days_after_op'])
# op_df_sum_chart_avg = op_df_sum_chart.groupby('launch_date').agg('days_after_op':'avg','total_app_tvl',:'sum')
# op_df_sum_chart_avg = op_df_sum_chart_avg.reset_index()
# op_launch_fig = px.bar(op_df_sum, x = 'launch_date', y = 'days_after_op',color = 'chain_id')
op_launch_fig = px.scatter(op_df_sum_chart, x = 'op_launch', y = 'days_after_op',color = 'chain',
                                trendline="ols", trendline_scope="overall", trendline_color_override="black"
                                ,labels={
                                "op_launch": "Date Launched on Optimism",
                                "days_after_op": "# Days Until Launched on Other Chain",
                                "chain": "Chain"
                                },
                                title="# Days Until App Launched on Other Chains (> $100k TVL) - Post OVM1 Mainnet\
                                <br><sup><i>Postive Days: App Launched on Optimism before Other Chains - Excluding Bridges</i></sup></br>"
                                ,hover_data=["protocol","launch_date"]
                                )


op_launch_fig.show()

op_df_sum_chart['time_period'] = np.where(op_df_sum_chart['op_launch'] < '2022-02-01','Pre-Feb 2022','>= Feb 2022')
# display( op_df_sum_chart.groupby(['time_period','op_status']).size() )
#remove unknown and unwanted tokens
# df_df = df_df[~df_df['token'].str.contains('UNKNOWN')]
# tl = df_df['token'].drop_duplicates()
# tl.to_csv('token_list.csv')
# display(df_df['token'].drop_duplicates())

In [ ]:
# ! jupyter nbconvert --to python launch_date.ipynb

In [ ]:


df_sum = df_df.copy()
#Filter Out L1
df_sum = df_sum[df_sum['chain'] != 'Ethereum']

df_sum = df_sum.groupby(['chain','protocol','date']).agg({'usd_value': 'sum','total_app_tvl':'mean'}) #flatten tokens - keep total TVL
# display(df_sum)
df_sum.reset_index(inplace=True)
df_sum = df_sum.groupby(['chain','protocol']).agg({'total_app_tvl': ['last','max'], 'date': 'min'})
df_sum.reset_index(inplace=True)

df_sum_cols = df_sum.columns.to_flat_index()
df_sum_cols
df_sum_cols = ['_'.join(filter(None, col)) for col in df_sum_cols]
df_sum.columns = df_sum_cols

# df_sum = df_sum.rename(columns={'usd_value_max':'max_usd_tvl','usd_value_last':'current_usd_tvl','date_min':'date'})
df_sum = df_sum.rename(columns={'total_app_tvl_max':'max_usd_tvl','total_app_tvl_last':'current_usd_tvl','date_min':'date'})
df_sum[df_sum['chain']=='Solana']

In [ ]:
display(df_sum)
df_sum = df_sum.merge(chains,on=['date','chain'],how='inner')
df_sum = df_sum.rename(columns={"date": "launch_date"})
df_sum = df_sum[df_sum['launch_date'].dt.date > start_date] #filter to trailing date

df_sum.sort_values('current_usd_tvl',ascending=True,inplace=True)
df_sum['chains_tvl_share_floor'] = np.floor(df_sum['chains_tvl_share'] / floor_round ) * floor_round
df_sum['chains_tvl_share_floor_sort'] = np.where(df_sum['chains_tvl_share_floor'] >= 50, 50, df_sum['chains_tvl_share_floor'])

df_sum['monthly_tvl_floor'] = np.floor(df_sum['totalLiquidityUSD'] / floor_round_tvl ) * floor_round_tvl
df_sum['monthly_tvl_share_floor'] = np.floor(df_sum['monthly_tvl_share'] / floor_round ) * floor_round
df_sum['monthly_tvl_share_floor_sort'] = np.where(df_sum['monthly_tvl_share_floor'] >= 30, 30, df_sum['monthly_tvl_share_floor'])

# df_sum['chains_tvl_share_floor'] = np.where(df_sum['chains_tvl_share_floor'] >= 50, '50+', round(df_sum['chains_tvl_share_floor']).astype(str))



# df_sum['num_protocols_threshold'] = np.where(df_sum['max_usd_tvl'] > tvl_protocol_threshold, 1, 0)
df_sum['num_protocols_threshold'] = np.where(df_sum['current_usd_tvl'] > tvl_protocol_threshold, 1, 0)
df_sum['num_protocols'] = 1
# df_sum['num_protocols_threshold_max'] = np.where(df_sum['current_usd_tvl'] < tvl_protocol_threshold_max, 1, 0)

df_sum['num_protocols_threshold_pct_chain'] = 100*df_sum['num_protocols_threshold'] / df_sum.groupby('chain')['num_protocols_threshold'].transform('sum')
df_sum['num_protocols_threshold_pct_share'] = 100*df_sum['num_protocols_threshold'] / df_sum.groupby('chains_tvl_share_floor')['num_protocols_threshold'].transform('sum')
# df_sum['num_protocols_threshold_max_pct_chain'] = np.where(df_sum['current_usd_tvl'] < tvl_protocol_threshold_max, 1, 0)

# display(df_sum[df_sum['chain']=='Polygon'].sort_values(by='chains_tvl_share_floor'))

display(df_sum[df_sum['chain']=='Arbitrum'].sort_values('launch_date',ascending=True))

In [ ]:
#FILTER DATE
df_sum.to_csv('tvl_data' + chain_lbl + '.csv')

In [ ]:
df_agg = df_sum.groupby(['chain','chains_tvl_share_floor']).agg(
                                                                {'current_usd_tvl':['sum','size']
                                                                ,'num_protocols_threshold':'sum'
                                                                # ,'num_protocols_threshold_max':'sum'
                                                                ,'num_protocols_threshold_pct_chain':'sum'
                                                                ,'num_protocols_threshold_pct_share':'sum'
                                                                })

df_agg = df_agg.reset_index()
df_cols = df_agg.columns.to_flat_index()
df_cols = ['_'.join(filter(None,col)) for col in df_cols]
df_agg.columns = df_cols
df_agg = df_agg.rename(columns={'current_usd_tvl_sum':'current_usd_tvl'
                                ,'current_usd_tvl_size':'num_protocols'
                                ,'num_protocols_threshold_sum':'num_protocols_threshold'
                                ,'num_protocols_threshold_max_sum':'num_protocols_threshold_max'
                                ,'num_protocols_threshold_pct_chain_sum':'num_protocols_threshold_pct_chain'
                                ,'num_protocols_threshold_pct_share_sum':'num_protocols_threshold_pct_share'}
                                )
# display(df_agg)

In [ ]:
chain_color_map={
                "Arbitrum": "#000000",
                "Avalanche": "pink",
                "Solana": "lightgreen",
                "Fantom": "#50ACE0",
                "Optimism": "red",
                "Polygon": "#7648D6",
                "Binance":"#EABC4E",
                "xDai":"darkgreen"
                }

In [ ]:

df_dt_add = df_sum.copy()
df_dt_add = df_dt_add[~df_dt_add['chain'].isin(['Optimism','Arbitrum','Fantom'])]
df_dt_add['launch_month'] = df_dt_add['launch_date'] + pd.offsets.MonthBegin(-1)
df_dt_add['launch_month_str'] = df_dt_add['launch_month'].astype(str)
df_dt_add['chain_id'] = df_dt_add['chain'] + ' - ' + df_dt_add['launch_month'].astype(str)

df_dt_add_share = df_dt_add.groupby(['chain_id','chain','launch_month','launch_month_str','monthly_tvl_share_floor_sort']).sum(['num_protocols','num_protocols_threshold','current_usd_tvl'])
df_dt_add_share.reset_index(inplace=True)

df_dt_add_tvl = df_dt_add.groupby(['chain_id','chain','launch_month','launch_month_str','monthly_tvl_floor']).sum(['num_protocols','num_protocols_threshold','current_usd_tvl'])
df_dt_add_tvl.reset_index(inplace=True)


# display(df_dt_add[df_dt_add['chain'] == 'Arbitrum'])
# display(df_dt_add.head())
fig_ch_scat = px.scatter(df_dt_add_share,
        x = 'monthly_tvl_share_floor_sort'
        # ,y = 'num_protocols'
        ,y = 'num_protocols_threshold'
        ,color = 'chain'
        ,title = '# of Protocols Deployed by Chain\'s TVL Share Each Month ($' + str(tvl_protocol_threshold/1e6) + 'M Min TVL)'
        ,hover_data=["launch_month_str"]
        ,color_discrete_map=chain_color_map
        ,trendline="ols", trendline_scope="trace"#, trendline_color_override="gray"
)
fig_ch_tvl = px.scatter(df_dt_add_tvl,
        x = 'monthly_tvl_floor'
        # ,y = 'num_protocols'
        ,y = 'num_protocols_threshold'
        ,color = 'chain'
        ,title = '# of Protocols Deployed by Chain\'s TVL Each Month ($' + str(floor_round_tvl/1e6) + 'M Buckets)'
        ,hover_data=["launch_month_str"]
        ,color_discrete_map=chain_color_map
        ,trendline="ols", trendline_scope="trace"#, trendline_color_override="gray"
)
fig_ch_tvl_size = make_subplots(specs=[[{"secondary_y": True}]])
fig_ch_tvl_size = px.scatter(
                df_dt_add_share,
                # df_dt_add_tvl,
        x = 'launch_month'
        # ,y = 'num_protocols'
        # ,y = 'monthly_tvl_floor'
        # ,y = 'monthly_tvl_share_floor_sort'
        # ,size = 'num_protocols_threshold'
        ,y = 'num_protocols_threshold'
        ,size = 'monthly_tvl_share_floor_sort'
        ,color = 'chain'
        ,title = '# of Protocols Deployed by Chain\'s TVL Each Month ($' + str(floor_round_tvl/1e6) + 'M Buckets)'
        ,hover_data=["launch_month_str"]
        ,color_discrete_map=chain_color_map
        # ,trendline="ols", trendline_scope="overall", trendline_color_override="gray"
)
# fig_ch_tvl_size.add_trace(go.Scatter(
#                 # df_dt_add_share,
#                 # df_dt_add_tvl,
#         x = df_dt_add_share['launch_month']
#         ,mode='markers'
#         # ,y = 'num_protocols'
#         # ,y = 'monthly_tvl_floor'
#         # ,y = 'monthly_tvl_share_floor_sort'
#         # ,size = 'num_protocols_threshold'
#         ,y = df_dt_add_share['num_protocols_threshold']
#         # ,size = df_dt_add_share['monthly_tvl_share_floor_sort']
#         # ,color = df_dt_add_share['chain']
#         # ,title = '# of Protocols Deployed by Chain\'s TVL Each Month ($' + str(floor_round_tvl/1e6) + 'M Buckets)'
#         # ,hover_data=[df_dt_add_share['launch_month_str']]
#         # ,color_discrete_map=chain_color_map
#         # ,trendline="ols", trendline_scope="overall", trendline_color_override="gray"

# ),secondary_y=False)
# fig_ch_tvl_size.add_trace(go.Line(x = all_chains['date'], y = all_chains['totalLiquidityUSD']),secondary_y=True)

fig_ch_scat.show()
fig_ch_tvl.show()
fig_ch_tvl_size.show()

fig_ch_tvl_size_go = make_subplots(specs=[[{"secondary_y": True}]])
fig_ch_tvl_size_go.add_trace(
    go.Scatter(x=[1, 2, 3], y=[40, 50, 60], name="yaxis data"),
    secondary_y=False,
)

fig_ch_tvl_size_go.add_trace(
    go.Scatter(x=[2, 3, 4], y=[4, 5, 6], name="yaxis2 data"),
    secondary_y=True,
)
display(df_dt_add_share)

In [ ]:
df_agg['chains_tvl_share_floor_sort'] = np.where(df_agg['chains_tvl_share_floor'] >= 30, 30, df_agg['chains_tvl_share_floor'])
df_agg_sort = df_agg.groupby(['chain','chains_tvl_share_floor_sort']).sum()
df_agg_sort.reset_index(inplace=True)
####
####
df_agg_sort = df_agg_sort.sort_values('chains_tvl_share_floor',ascending=True)
# display(df_agg)
fig = px.bar(   
                # x=df_agg['chains_tvl_share_floor'],
                x=df_agg_sort['chains_tvl_share_floor_sort'],
                # y=df_agg['current_usd_tvl'],
                # y=df_agg['num_protocols'],
                y=df_agg_sort['num_protocols_threshold'],
                
                # y = df_agg['num_protocols_threshold_max'],
                color = df_agg_sort['chain'],
                color_discrete_map = chain_color_map,
                labels={'x':'Chain\'s % Share of TVL (Overflow 30%+)', 'y':'# Protocols'},
                title= str(np.where(
                        trailing_num_days <= 9999,'Last ' + str(trailing_num_days) + ' Days',
                        'All-Time'
                        ))
                        +' - Protocols Launched by Chain\'s Share of TVL (Min $' 
                        + str(tvl_protocol_threshold/1e6) + 'M Max App TVL)')
fig.update_xaxes(categoryorder='category ascending')
fig.show()

# fig_s = px.bar(
#                 # x=df_agg['chains_tvl_share_floor'],
#                 x=df_agg['chains_tvl_share_floor_sort'],
#                 # y=df_agg['current_usd_tvl'],
#                 # y=df_agg['num_protocols'],
#                 y=df_agg['num_protocols_threshold_pct_share'],
#                 # y = df_agg['num_protocols_threshold_max'],
#                 color = df_agg['chain'],
#                 color_discrete_map = chain_color_map,
#                 labels={'x':'Chain\'s % Share of TVL', 'y':'# Protocols'},
#                 title= str(np.where(
#                         trailing_num_days <= 9999,'Last ' + str(trailing_num_days) + ' Days',
#                         'All-Time'
#                         ))
#                         +' - Protocols Launched by Chain\'s Share of TVL (Min $' 
#                         + str(tvl_protocol_threshold/1e6) + 'M Max App TVL)')
# fig_s.update_xaxes(categoryorder='category ascending')
# fig_s.show()


fig_ch = px.bar(x=df_agg['chain'],
                # y=df_agg['current_usd_tvl'],
                # y=df_agg['num_protocols'],
                y=df_agg['num_protocols_threshold'],
                # y = df_agg['num_protocols_threshold_max'],
                color = df_agg['chains_tvl_share_floor'],
                labels={'x':'Chain', 'y':'# Protocols','color':'Chain\'s % Share of TVL'},
                title= str(np.where(
                        trailing_num_days <= 9999,'Last ' + str(trailing_num_days) + ' Days',
                        'All-Time'
                        ))
                        +' - Protocols Launched by Chain\'s Share of TVL (Min $' 
                        + str(tvl_protocol_threshold/1e6) + 'M Max App TVL)',
                # color_discrete_sequence=px.colors.sequential.Blues
                color_continuous_scale= 'Blues'
                )

fig_ch.show()

display(df_agg.groupby('chain').sum())


In [ ]:
# print(pd.Timestamp.today().floor('d') + pd.offsets.MonthBegin(-1))
df_agg_dt = df_sum.copy()
# display(df_agg_dt)
df_agg_dt = df_agg_dt[df_agg_dt['launch_date'] > '2021-05-01']
# df_agg_dt = df_agg_dt[df_agg_dt['launch_date'] >= '2020-01-01']
df_agg_dt['month'] = df_agg_dt['launch_date'] + pd.offsets.MonthBegin(-1)
df_agg_dt = df_agg_dt[df_agg_dt['month'] < (pd.Timestamp.today().floor('d') + pd.offsets.MonthBegin(-1))]
df_agg_dt = df_agg_dt[['month','chains_tvl_share_floor','num_protocols_threshold','num_protocols','chains_tvl_share_floor_sort']].groupby(['month','chains_tvl_share_floor','chains_tvl_share_floor_sort']).sum(['num_protocols_threshold','num_protocols'])
df_agg_dt.reset_index(inplace=True)
df_agg_dt['num_protocols_threshold_pct'] = 100*df_agg_dt['num_protocols_threshold'] / df_agg_dt.groupby('month')['num_protocols_threshold'].transform('sum')
df_agg_dt['num_protocols_pct'] = 100*df_agg_dt['num_protocols'] / df_agg_dt.groupby('month')['num_protocols'].transform('sum')

df_agg_dt['chains_tvl_share_floor'] = df_agg_dt['chains_tvl_share_floor']#.astype(str)
# df_agg_dt = df_agg_dt.sort_values(by=['chains_tvl_share_floor_sort','month'],ascending=[True,True])
df_agg_dt = df_agg_dt.sort_values(by=['chains_tvl_share_floor','month'],ascending=[True,True])
display(df_agg_dt)
fig_ln = px.bar(df_agg_dt, x='month', y='num_protocols_threshold_pct',color='chains_tvl_share_floor'
        # ,color_discrete_sequence=px.colors.sequential.Blues
        ,color_continuous_scale= 'Blues'
        ,labels={
                "month": "Month",
                "num_protocols_threshold_pct": "% of App's Deployed (Above Threshold)",
                "chains_tvl_share_floor": "% of In-Scope Chain's TVL"
                },
                title="Share of Monthly New Apps by Chain\'s TVL Share\
                <br><sup><i>(Min $" + str(tvl_protocol_threshold/1e6) + "M Max App TVL) " +\
                         str(np.where(filter_to_chains==1, str(chains_list), ' - All Chains')) +\
                                 "</i></sup></br>"
        )

fig_ln.show()

total_agg = df_agg_dt[['chains_tvl_share_floor','num_protocols_threshold']].groupby('chains_tvl_share_floor').sum()
total_agg = total_agg.reset_index().sort_values(by='chains_tvl_share_floor',ascending=True)
display(total_agg)
# fig_ttl = px.pie(total_agg, values='num_protocols_threshold_pct', names='chains_tvl_share_floor', title="Share of Apps Deployed by Chain TVL Share\
#                 <br><sup><i>(Min $" + str(tvl_protocol_threshold/1e6) + "M Max App TVL) " +\
#                          str(np.where(filter_to_chains==1, str(chains_list), ' - All Chains')) +\
#                                  "</i></sup></br>"
#                 ,color_discrete_sequence= px.colors.sequential.Blues
#                 , hole=.3
#                 , category_orders=ascending
#                                  )
fig_ttl = go.Figure(
    data=[go.Pie(
        labels=total_agg['chains_tvl_share_floor'],
        values=total_agg['num_protocols_threshold'],
        # Second, make sure that Plotly won't reorder your data while plotting
        
        # ,color_discrete_sequence= px.colors.sequential.Blues
        marker_colors=px.colors.sequential.Blues
        ,hole=0.3
        ,sort=False)
    ])
fig_ttl.update_traces(textposition='inside', textinfo='percent+label')
fig_ttl.update_layout(title="Share of Apps Deployed by Chain\'s TVL Share <br><sup><i>(Min $" + str(tvl_protocol_threshold/1e6) + "M Max App TVL) " +\
                         str(np.where(filter_to_chains==1, str(chains_list), ' - All Chains')) +\
                                 "</i></sup></br>")
fig_ttl.show()

In [ ]:


fig_pct = px.bar(x=df_agg['chains_tvl_share_floor'],
                # y=df_agg['current_usd_tvl'],
                # y=df_agg['num_protocols'],
                y=df_agg['num_protocols_threshold_pct_chain'],
                # y = df_agg['num_protocols_threshold_max'],
                color = df_agg['chain'],
                labels={'x':'Chain\'s % Share of TVL', 'y':'# of Protocols'},
                title= str(np.where(
                        trailing_num_days <= 9999,'Last ' + str(trailing_num_days) + ' Days',
                        'All-Time'
                        ))
                        +' - Protocols Launched by Chain\'s Share of TVL (Min $'  
                        + str(tvl_protocol_threshold/1e6) + 'M Max App TVL)')

fig_pct.show()

fig_ch_pct = px.bar(x=df_agg['chain'],
                # y=df_agg['current_usd_tvl'],
                # y=df_agg['num_protocols'],
                y=df_agg['num_protocols_threshold_pct_chain'],
                # y = df_agg['num_protocols_threshold_max'],
                color = df_agg['chains_tvl_share_floor'],
                labels={'x':'Chain', 'y':'% of Protocols','color':'Chain\'s % Share of TVL'},
                title= str(np.where(
                        trailing_num_days <= 9999,'Last ' + str(trailing_num_days) + ' Days',
                        'All-Time'
                        ))
                        +' - Protocols Launched by Chain\'s Share of TVL (Min $' 
                        + str(tvl_protocol_threshold/1e6) + 'M Max App TVL)'
                # ,color_discrete_sequence=px.colors.sequential.Blues
                ,color_continuous_scale= 'Blues'
                )

fig_ch_pct.show()





In [ ]:
tvl_fig = px.bar(x=df_agg['chains_tvl_share_floor'],
                y=df_agg['current_usd_tvl'],
                # y=df_agg['num_protocols'],
                # y=df_agg['num_protocols_threshold'],
                # y = df_agg['num_protocols_threshold_max'],
                color = df_agg['chain'],
                labels={'x':'Chain\'s % Share of TVL', 'y':'Current TVL'},
                title= str(np.where(
                        trailing_num_days <= 9999,'Last ' + str(trailing_num_days) + ' Days',
                        'All-Time'
                        ))
                        +' - Protocols Launched by Chain\'s Share of TVL (Min $' 
                        + str(tvl_protocol_threshold/1e6) + 'M Max App TVL)')

tvl_fig.show()

tvl_fig_ch = px.bar(x=df_agg['chain'],
                y=df_agg['current_usd_tvl'],
                # y=df_agg['num_protocols'],
                # y=df_agg['num_protocols_threshold'],
                # y = df_agg['num_protocols_threshold_max'],
                color = df_agg['chains_tvl_share_floor'],
                labels={'x':'Chain', 'y':'Current TVL','color':'Chain\'s % Share of TVL'},
                title= str(np.where(
                        trailing_num_days <= 9999,'Last ' + str(trailing_num_days) + ' Days',
                        'All-Time'
                        ))
                        +' - Protocols Launched by Chain\'s Share of TVL (Min $' 
                        + str(tvl_protocol_threshold/1e6) + 'M Max App TVL)'
                ,color_discrete_sequence=px.colors.sequential.Blues
                )

tvl_fig_ch.show()

In [ ]:

fig_share = px.pie(current_tvl, values='totalLiquidityUSD', names='chain'
        , title='Current Chain Share of TVL<br><sup><i>Includes \"Double Count\" Protocols</i></sup>'
        , hole=.3)
fig_share.update_traces(textposition='inside', textinfo='percent+label')
fig_share.show()

# display(chains)
fig_chains = px.line(chains, x="date", y="chains_tvl_share", color = 'chain'
        , title='Share of TVL by Chain<br><sup><i>Includes \"Double Count\" Protocols</i></sup>'
        ,color_discrete_map=chain_color_map)
fig_chains.show()

In [ ]:
df_count = df_sum.copy()
# display(df_count[df_count['chain'] == 'Polygon'])
# df_count = df_count[['chain','protocol','current_tvl']]
# df_count = df_count.drop_duplicates()
df_count = df_count.groupby('chain').agg({'num_protocols_threshold':'sum','num_protocols':'sum','current_usd_tvl':'sum'})
# df_count = df_count.rename(columns={'0','num_protocols_threshold'})
df_count = df_count.reset_index()
display(df_count)



fig_ct_share = px.pie(df_count, values='num_protocols_threshold', names='chain'
        , title='# of Protocols by Chain ($' + str(tvl_protocol_threshold/1e6) + 'M Min TVL) <br><sup><i>Includes \"Double Count\" Protocols</i></sup>'
        , hole=.3
        ,color_discrete_map=chain_color_map
        )
fig_ct_share.update_traces(textposition='inside', textinfo='percent+value+label')
fig_ct_share.show()

####

fig_ct_share_sct = px.scatter(df_count, y='num_protocols_threshold', x='current_usd_tvl', color = 'chain'
        , title='# of Protocols by Chain ($' + str(tvl_protocol_threshold/1e6) + 'M Min TVL) <br><sup><i>Includes \"Double Count\" Protocols</i></sup>'
        # , hole=.3
        ,color_discrete_map=chain_color_map
        ,trendline="ols", trendline_scope="overall", trendline_color_override="gray"
        ,labels={
                                "current_usd_tvl": "Chain TVL (USD)",
                                "num_protocols_threshold": "Dapps",
                                "chain": "Chain"
                                },
        # ,marker_size=4
        width=800, height=600)
# fig_ct_share.update_traces(textposition='inside', textinfo='percent+value+label')
fig_ct_share_sct.update_traces(marker={'size': 10})
fig_ct_share_sct.show()

In [ ]:
#sol, poly, bnb, arbi
clist = ['Solana','Polygon','Binance','Arbitrum','Optimism']



for ci in clist:
        tmp = df_sum[df_sum['chain'] == ci]
        # tmp = df_sum[df_sum['chain'] == 'Arbitrum']
        tmp = tmp[tmp['num_protocols_threshold'] > 0]
        fig_h = px.histogram(tmp, x="current_usd_tvl",# nbins = 50,
                labels={
                        "current_usd_tvl":'Current App TVL (USD)',
                        "count":"# Protocols"
                },width=800, height=400).update_layout(
                title={"text": ci + " - # Protocols Distribution by TVL ($" + str(tvl_protocol_threshold/1e6) + "M Min TVL)"}, yaxis_title="# Protocols"
                )
        fig_h.show()
        # display(tmp)
# for ci in clist:
#         tmp = df_sum[df_sum['chain'] == ci]
#         # tmp = df_sum[df_sum['chain'] == 'Arbitrum']
#         tmp = tmp[tmp['num_protocols_threshold'] > 0]
#         fig_h = px.histogram(tmp, x="current_usd_tvl",# nbins = 50,
#                 labels={
#                         "current_usd_tvl":'Current App TVL (USD)',
#                         "count":"# Protocols"
#                 },width=800, height=400).update_layout(
#                 title={"text": ci + " - # Protocols Distribution by TVL ($" + str(tvl_protocol_threshold/1e6) + "M Min TVL)"}, yaxis_title="# Protocols"
#                 )
#         fig_h.show()
#         # display(tmp)